In [1]:
import logging
import pandas as pd
from sklearn.model_selection import train_test_split
import os.path
from os import path
import pandas as pd
import numpy as np
import base64
import re
import matplotlib.pyplot as plt

In [56]:
from transformers import TFBertModel,  BertConfig, BertTokenizerFast

# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model, save_model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy,BinaryCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy,BinaryAccuracy
from tensorflow.keras.utils import to_categorical
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Sequential

In [3]:
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
file_name = "poems_with_new_tags.zip"
poem_df = pd.read_csv('poems_with_new_tags.zip', compression='zip', header=0, quotechar='"')


In [5]:
poems_info = poem_df.drop(['star','author_stars','tags','tags_list','new_tags','new_first_tag'], axis=1)

In [6]:
poems_info.head()

,content,dynasty,author,title,写物,劝勉,家庭,快乐,悲苦,政治,朋友,游玩
0,《吴都赋》云：“户藏烟浦，家具画船。”唯吴兴为然。春游之盛，西湖未能过也。己酉岁，予与萧时父...,宋代,姜夔,琵琶仙·《吴都赋》云：「户藏烟浦,0,0,0,0,0,1,0,1
1,《廿一史弹词》第三段说秦汉开场词滚滚长江东逝水，浪花淘尽英雄。是非成败转头空。青山依旧在，几...,明代,杨慎,临江仙·滚滚长江东逝水,1,1,0,0,1,1,0,0
2,《水经》云：“彭蠡之口有石钟山焉。”郦元以为下临深潭，微风鼓浪，水石相搏，声如洪钟。是说也，...,宋代,苏轼,石钟山记,1,0,0,0,0,1,0,1
3,【序】辛亥之冬，予载雪诣石湖。止既月，授简索句，且征新声，作此两曲。石湖把玩不已，使工妓隶习...,宋代,姜夔,暗香疏影,0,1,0,0,0,0,0,0
4,〔一枝花〕 攀出墙朵朵花，折临路枝枝柳。花攀红蕊嫩，柳折翠条柔，浪子风流。凭着我折柳攀花手...,元代,关汉卿,【南吕】一枝花不伏老,1,0,0,1,1,0,0,0


In [30]:
poems_info.iloc[:, 4:].values

array([[0, 0, 0, ..., 1, 0, 1],
       [1, 1, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 1, 0, 1],
       ...,
       [1, 0, 0, ..., 1, 1, 1],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 0, 0]])

In [7]:
OUTPUT_COLUMSN=['写物', '劝勉', '家庭', '快乐', '悲苦', '政治', '朋友', '游玩']

In [8]:
data, data_test = train_test_split(poems_info, test_size = 0.2)

In [10]:
# Name of the BERT model to use
model_name = 'bert-base-chinese'
# Max length of tokens
max_length = 100
# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False
# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
# Load the Transformers BERT model
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

Some layers from the model checkpoint at bert-base-chinese were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-chinese.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [52]:
bert = transformer_model.layers[0]
# Build your model input
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
# input_mask = Input(shape=(max_length,), name='input_mask', dtype='int32')
# segment_ids = Input(shape=(max_length,), name='segment_ids', dtype='int32')

inputs = {'input_ids': input_ids}
#, 'input_mask': input_mask, 'segment_ids':segment_ids}
# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[0]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)


LSTM_Layer_1 = LSTM(128)(pooled_output)
dense_layer_1 = Dense(8, activation='sigmoid')(LSTM_Layer_1)
model = Model(inputs=inputs, outputs=dense_layer_1)



In [63]:
optimizer = Adam(
    learning_rate=0.05,
    epsilon=1e-08,
    decay=0.1,
    clipnorm=1.0)

In [64]:
model.compile(loss=BinaryCrossentropy(from_logits = False), optimizer=optimizer, metrics=[BinaryAccuracy('accuracy')])

In [65]:
model.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 100)]             0         
_________________________________________________________________
bert (TFBertMainLayer)       TFBaseModelOutputWithPool 102267648 
_________________________________________________________________
pooled_output (Dropout)      (None, 100, 768)          0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 128)               459264    
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 1032      
Total params: 102,727,944
Trainable params: 102,727,944
Non-trainable params: 0
_________________________________________________________________


In [66]:
# Tokenize the input (takes some time)
x = tokenizer(
    text=data['content'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)
# Prepare the y
y = poems_info.iloc[:, 4:].values

In [ ]:
# Fit the model
history = model.fit(
    x=x['input_ids'],
    y=y,
    validation_split=0.2,
    batch_size=32,
    epochs=2)

Epoch 1/2


04/11/2021 01:49:55 - WARNING - tensorflow -   Gradients do not exist for variables ['tf_bert_model/bert/pooler/dense/kernel:0', 'tf_bert_model/bert/pooler/dense/bias:0'] when minimizing the loss.


04/11/2021 01:49:57 - WARNING - tensorflow -   Gradients do not exist for variables ['tf_bert_model/bert/pooler/dense/kernel:0', 'tf_bert_model/bert/pooler/dense/bias:0'] when minimizing the loss.


04/11/2021 01:50:11 - WARNING - tensorflow -   Gradients do not exist for variables ['tf_bert_model/bert/pooler/dense/kernel:0', 'tf_bert_model/bert/pooler/dense/bias:0'] when minimizing the loss.


04/11/2021 01:50:13 - WARNING - tensorflow -   Gradients do not exist for variables ['tf_bert_model/bert/pooler/dense/kernel:0', 'tf_bert_model/bert/pooler/dense/bias:0'] when minimizing the loss.


123/123 [==============================] - 1932s 16s/step - loss: 0.5552 - accuracy: 0.7188 - val_loss: 0.5414 - val_accuracy: 0.7114
Epoch 2/2
 95/123 [======================>.......] - ETA: 6:35 - loss: 0.5416 - accuracy: 0.7242

In [44]:
x['token_type_ids']

<tf.Tensor: shape=(4885, 100), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>